# TensorFlow CNN - NLP

In [ ]:
import tensorflow as tf
import pandas as pd

## Get IMdb data

In [ ]:
# 50,000 movie reviews labeled as "positive" or "negative"
imdb_data = pd.read_csv("IMDB Dataset.csv")

# Binary transformation of target variable
imdb_data["sentiment"] = imdb_data["sentiment"].map({"positive": 1, "negative":0})

# Train/Test Split
imdb_train = imdb_data["review"].values[:25000]
imdb_test = imdb_data["review"].values[25000:]

imdb_train_labels = imdb_data["sentiment"].values[:25000]
imdb_test_labels = imdb_data["sentiment"].values[25000:]

## Tokenize

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000, oov_token = "<oov>")
tokenizer.fit_on_texts(imdb_train)
word_index = tokenizer.word_index

# convert words to numbers and pad for the neural network to use as input
# Training data
train_sequences = tokenizer.texts_to_sequences(imdb_train)
train_padded = pad_sequences(train_sequences, maxlen=120, truncating="post")

# Test data
testing_sequences = tokenizer.texts_to_sequences(imdb_test)
test_padded = pad_sequences(testing_sequences, maxlen=120, truncating="post")

## Neural Network

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 64, input_length=120),
    tf.keras.layers.Flatten(),
    #tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(train_padded, imdb_train_labels, validation_split=.1, epochs=10)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+ string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()
             
plot_graphs(history,'accuracy')
plot_graphs(history, 'loss')

## Convolutional Neural Network

In [ ]:

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 64, input_length=120),
    tf.keras.layers.Conv1D(128, 3, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(16, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

### Train the model

In [ ]:
history = model.fit(train_padded, imdb_train_labels, validation_split=.1, epochs=5)

### Make predictions on text samples

In [ ]:
# predict on a positive sample

sample_text_to_predict = ["The movie was cool. The animation and the graphics were out of this world. I would recommend this movie."]

pos_sequence = tokenizer.texts_to_sequences(sample_text_to_predict)
pos_padded = pad_sequences(pos_sequence, maxlen=120, truncating="post")

#  make prediction
prediction = model.predict(pos_padded)

print(prediction)

In [ ]:
# predict on a negative sample

sample_text_to_predict = ["The movie was not good. The animation and the graphics were terrible. I would not recommend this movie."
]

neg_sequence = tokenizer.texts_to_sequences(sample_text_to_predict)
neg_padded = pad_sequences(neg_sequence, maxlen=120, truncating="post")

#  make prediction
prediction = model.predict(neg_padded)

print(prediction)

### Plot learning curves

In [ ]:
plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')